In [18]:
import pandas as pd
import sqlite3 as sql

In [19]:
con =sql.connect('/content/sql-murder-mystery.db')


*RETRIEVE CRIME SCENE REPORT*

In [20]:
query_1="""
SELECT * FROM crime_scene_report
WHERE city='SQL City'
and date=20180115
and type='murder';
"""
pd.set_option('display.max_colwidth',None)
pd.read_sql_query(query_1,con)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


*WITNESS PERSONAL DETAILS:*

In [21]:
query_2="""
SELECT *
FROM Person
WHERE address_street_name='Northwestern Dr'
ORDER BY address_number desc;
"""
pd.read_sql_query(query_2,con)[:1]

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [22]:
query_3="""
SELECT *
FROM Person
WHERE address_street_name='Franklin Ave' and name like 'Annabel%'
"""
pd.read_sql_query(query_3,con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


*VIEW WITNESS INTERVIEWS*

In [23]:
query_4="""
SELECT *
FROM interview
WHERE person_id=16371 or person_id=14887;
"""
pd.read_sql_query(query_4,con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


*CHECK GYM DATABASE*

In [24]:
query_5="""
SELECT *
FROM get_fit_now_member
WHERE id LIKE '48Z%' and membership_status='gold';
"""
pd.read_sql_query(query_5,con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [25]:
query_6="""
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date=20180109 and membership_id LIKE '48Z%';
"""
pd.read_sql_query(query_6,con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


*CHECK CAR DETAILS*

In [26]:
query_7="""
SELECT
* FROM drivers_license
WHERE plate_number LIKE '%H42W%';
"""
pd.read_sql_query(query_7,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


*PERSONAL DETAILS *

In [27]:
query_8="""
SELECT
* FROM person
WHERE license_id=183779 or license_id=423327 or license_id=664760;
"""
pd.read_sql_query(query_8,con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


*MEMBERSHIP STATUS AT GYM*

In [28]:
query_9="""
SELECT *
FROM person
WHERE (license_id=183779 or license_id=423327 or license_id=664760) AND
(id=28819 or id=67318);
"""
pd.read_sql_query(query_9,con)

,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [29]:
query_10="""
SELECT *
FROM interview
WHERE person_id=67318;
"""
pd.read_sql_query(query_10,con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


*THE MURDERER IS Jeremy Bowers
Both the membership id and status also matches as per the information we found earlier*

*But after we check the interview of Jeremy Bowers we found that he was hired by a women.*

In [32]:
query_10="""
SELECT *
FROM interview
WHERE person_id=67318;
"""
pd.read_sql_query(query_10,con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [33]:
query_11="""
SELECT *
FROM drivers_license
WHERE gender='female' and height>=65 and hair_color='red' and car_make='Tesla'and car_model='Model S';
"""
pd.read_sql_query(query_11,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


Three woman with Tesla Model S and red hair color and same height.

In [34]:
query_12="""
SELECT person_id, count(*), event_name
FROM facebook_event_checkin
GROUP BY person_id
having count(*) = 3 AND event_name = "SQL Symphony Concert" AND date like "201712%";
"""
pd.read_sql_query(query_12,con)

,person_id,count(*),event_name
0,24556,3,SQL Symphony Concert
1,99716,3,SQL Symphony Concert


In [35]:
query_13="""
SELECT * FROM person
WHERE id=24556 or id=99716;
"""
pd.read_sql_query(query_13,con)

,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [36]:
query_14="""
SELECT * FROM drivers_license
WHERE id=101191 or id=202298;
"""
pd.read_sql_query(query_14,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S


*FINALLY, WE FOUND THE REAL MASTER MIND OF THIS MURDER MYSTERY WHICH IS - Miranda Priestly*